In [1]:
import pandas as pd
from tqdm import tqdm
from pororo import Pororo

[Korean Sentence Splitter]: Initializing Kss...


In [17]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
wiki_df = pd.read_json('/opt/ml/data/wikipedia_documents.json', orient = 'index')
wiki_df.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,NaN,나라 목록,NaN,NaN,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,NaN,나라 목록,NaN,NaN,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,NaN,백남준,NaN,NaN,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,NaN,아오조라 문고,NaN,NaN,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,NaN,아오조라 문고,NaN,NaN,4


In [8]:
wiki_df.to_csv('wiki_df.csv', index=False)

In [18]:
qg = Pororo(task="qg", lang="ko")
qg(
    "카카오톡",
    "카카오톡은 스마트폰의 데이터 통신 기능을 이용하여, 문자 과금 없이 사람들과 메시지를 주고받을 수 있는 애플리케이션이다. 스마트폰 대중화 이 후 기존 인스턴트 메신저 앱의 번거로운 친구 추가 절차 없이, 스마트폰 주소록의 전 화번호만으로 손쉽게 메시지를 주고받을 수 있는 것이 특징이다.",
    n_wrong=3
)

('스마트폰의 데이터 통신 기능을 이용하여, 문자 과금 없이 사람들과 메시지를 주고받을 수 있는 애플리케이션 이름은?',
 ['텔레그램', '챗온', '디스코드'])

In [43]:
len(wiki_df['title'])

60613

In [44]:
len(wiki_df['title'].unique())

31755

In [40]:
wiki_df['title'][147]

'영어'

In [41]:
wiki_df['text'][147]

'영어권\n 근대 잉글랜드어는 종종 최초의 지구적 링구아 프랑카로서 언급된다.   잉글랜드어는 통신, 과학, 무역, 비행, 오락, 방송, 외교 등의 분야에서 국제어로서 사용되고 있다. 때문에 잉글랜드어의 사용은 종종 언어 제국주의라는 비판을 받기도 한다.  잉글랜드어는 대영제국의 팽창과 함께 브리튼 제도의 밖에서 사용되기 시작하였으며 19세기 말 무렵에는 글자 그대로 전지구적 언어가 되었다.  영국의 식민지였던 미국과 캐나다 역시 잉글랜드어 사용의 확대에 기여하였는데, 특히 제2차 세계대전 이후 초강국이 된 미국으로 인해 잉글랜드어 사용의 세계화는 더욱 확산되었다\n\n의료나 컴퓨터 사용과 같은 여러 분야는 기초적인 잉글랜드어 사용 능력을 전제로 한다. 이를 위해 수억명 이상의 사람들이 잉글랜드어를 배운다.  잉글랜드어는 유엔의 여섯 공용어 가운데 하나이다.\n\n영국의 언어학자 데이비드 크리스털은 전 지구적인 잉글랜드어 사용의 급격한 증가로 인해 다른 언어가 막대한 타격을 입고 있다고 말한다. 북아메리카와 오스트레일리아를 포함한 지구 곳곳에서 잉글랜드어 사용이 일반화됨으로써 각 지역의 자연어가 갖고 있던 언어 다양성이 감소하고 심지어 더이상 쓰는 사람이 없는 사어가 되고 있다고 지적한다.  이러한 잉글랜드어의 영향은 역사언어학의 연구에서도 확인할 수 있다. 여러 언어 공동체에 광범위하게 확산되어 사용되고 있는 잉글랜드어는 복잡하고 역동적인 언어 변화를 가져왔다. 크리올과 피진은 이러한 잉글랜드어의 영향으로 만들어진 새로운 어족이다.'

In [42]:
qg(wiki_df['title'][147], wiki_df['text'][147])

'잉글랜드어는 어느 나라의 공용어야?'

In [274]:
contexts = []
questions = []
answers = []
doc_ids = []
error_ids = []
for i in tqdm(range(len(wiki_df))):
    c = wiki_df['text'][i]
    a = wiki_df['title'][i]
    if a not in answers and c not in contexts:
        try:
            q = qg(a, c, beam = 10, top_p=0.9)
        except:
            error_ids.append(i)
            continue
        contexts.append(c)
        answers.append(a)
        questions.append(q)
        doc_ids.append(wiki_df['document_id'][i])
wiki_qg_arg_df = pd.DataFrame(data={'answer':answers,'question':questions,'context':contexts, 'doc_id':doc_ids})
wiki_qg_arg_df.to_csv('wiki_qg_arg_df.csv', index=False)

100%|██████████| 60613/60613 [4:57:42<00:00,  3.39it/s]


In [46]:
wiki_qa_df

,answer,question,context,doc_id
0,나라 목록,전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하는 문서의 ...,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",0
1,나라 목록,ISIS는 준국가로 분류되는 이유가 뭐야?,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,1
2,백남준,1966년 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양...,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,2
3,아오조라 문고,일본어판 구텐베르크 프로젝트로 불리는 일본의 인터넷 전자도서관 이름은?,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",3
4,아오조라 문고,일본 작가들의 작품은 얼마나 있어?,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,4
...,...,...,...,...
56691,오키나와 현립 박물관·미술관,오키나와에 있는 최초의 복합 문화시설은?,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",60608
56692,오키나와 현립 박물관·미술관,오키나와 민정부립 슈리 박물관이 바뀐 건 언제야?,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",60609
56693,오키나와 현립 박물관·미술관,박물관이 관의 애칭 및 마스코트를 공모한게 언제야?,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,60610
56694,단결권 및 단체교섭권 협약,국제 노동 기구의 8대 핵심 협약 중 하나가 뭐야?,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,60611


In [275]:
wiki_qg_arg_df

,answer,question,context,doc_id
0,나라 목록,세계 각 지역의 현황과 주권 승인 정보를 기재하는 문서의 이름은?,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",0
1,백남준,"""큰대문집""에서 18세까지 살고, 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 ...",현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,2
2,아오조라 문고,일본어판 구텐베르크 프로젝트로 불리는 일본의 인터넷 전자도서관 이름은?,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",3
3,데니스 리치,애플 컴퓨터는 누구의 업적을 기반으로 하고 있어?,켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발...,8
4,주기율표,원소를 구분하기 쉽게 성질에 따라 배열한 표가 뭐야?,"주기율표(週期律表, 주기률표, periodic table) 또는 주기표(週期表)는 ...",9
...,...,...,...,...
31705,2022년 FIFA 월드컵 유럽 지역 예선 플레이오프,결승전에서 승리한 3개 팀은 언제 본선 진출이 가능한가?,2022년 FIFA 월드컵 유럽 지역 예선 플레이오프 진행 방식은 2019년 12월...,60597
31706,자주국방론,박정희 정부가 분쟁의 책임을 당사국에 국한시킨 주장이 뭐야?,"박정희 정부가 자주국방론을 내놓은 배경은 ""아시아의 공산주의 국가와 자유진영 국가의...",60598
31707,페름의 문장,페름에 사는 곰은 어디에 살아?,"곰 자체는 페름 지역 숲에 사는 동물이 풍부함을 상징하며, 은색인 이유는 금속, 소...",60606
31708,오키나와 현립 박물관·미술관,오키나와에 있는 최초의 복합 문화시설은?,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",60608


In [277]:
wiki_qg_arg_df['answer'][31707] in wiki_qg_arg_df['context'][31707]

False

In [279]:
contexts = []
questions = []
answers = []
doc_ids = []
error_ids = []
flag = True
for i in tqdm(range(len(wiki_qg_arg_df))):
    if wiki_qg_arg_df['answer'][i] in wiki_qg_arg_df['question'][i]:
        continue
    elif not wiki_qg_arg_df['answer'][i] in wiki_qg_arg_df['context'][i]:
        continue
    contexts.append(wiki_qg_arg_df['context'][i])
    answers.append(wiki_qg_arg_df['answer'][i])
    questions.append(wiki_qg_arg_df['question'][i])
    doc_ids.append(wiki_qg_arg_df['doc_id'][i])
wiki_arg_post_processing = pd.DataFrame(data={'answer':answers,'question':questions,'context':contexts, 'doc_id':doc_ids})

100%|██████████| 31710/31710 [00:01<00:00, 28691.68it/s]


In [ ]:
wiki_arg_post_processing.to_csv('wiki_arg_post_processing.csv', index=False)

In [282]:
wiki_arg_post_processing

,answer,question,context,doc_id
0,나라 목록,세계 각 지역의 현황과 주권 승인 정보를 기재하는 문서의 이름은?,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",0
1,백남준,"""큰대문집""에서 18세까지 살고, 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 ...",현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,2
2,아오조라 문고,일본어판 구텐베르크 프로젝트로 불리는 일본의 인터넷 전자도서관 이름은?,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",3
3,데니스 리치,애플 컴퓨터는 누구의 업적을 기반으로 하고 있어?,켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발...,8
4,주기율표,원소를 구분하기 쉽게 성질에 따라 배열한 표가 뭐야?,"주기율표(週期律表, 주기률표, periodic table) 또는 주기표(週期表)는 ...",9
...,...,...,...,...
16525,타바투이호,네이바가 어느 호수에서 태어났어?,타바투이호의 형성과 관련하여 전해지는 전설이 있다.\n\n예전에 이 지역에는 아름답...,60594
16526,2022년 FIFA 월드컵 유럽 지역 예선 플레이오프,결승전에서 승리한 3개 팀은 언제 본선 진출이 가능한가?,2022년 FIFA 월드컵 유럽 지역 예선 플레이오프 진행 방식은 2019년 12월...,60597
16527,자주국방론,박정희 정부가 분쟁의 책임을 당사국에 국한시킨 주장이 뭐야?,"박정희 정부가 자주국방론을 내놓은 배경은 ""아시아의 공산주의 국가와 자유진영 국가의...",60598
16528,오키나와 현립 박물관·미술관,오키나와에 있는 최초의 복합 문화시설은?,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",60608


In [283]:
wiki_arg_post_processing['context'][16525]

'타바투이호의 형성과 관련하여 전해지는 전설이 있다.\n\n예전에 이 지역에는 아름답고 강한 사람들이 살았는데, 특히 토솀의 딸 네이바가 아름답기로 소문이 나 부족의 모든 남자들이 비싼 몸값을 지불하며 결혼을 구애했으나, 젊은 사냥꾼 타바투이만이 오지 않았으며, 네이바는 타바투이를 기다리며 시간이 흘렀다. 하지만 어느 날 동쪽에서 온 사람들이 부족을 약탈하기 시작하자 부족의 주술사는 숲 속 깊이 들어가기로 결정하였으며, 타바투이만이 반대하여 침략자와 맞서 싸울 것을 주장하며 전투가 벌어졌다. 전투는 일출부터 일몰까지 이어졌으며 결국 타바투이는 피를 흘리며 쓰러졌다. 네이바는 타바투이의 죽음을 보며 울음을 터트렸고, 이 눈물이 모여 타바투이 위를 덮으며 호수가 만들어졌다. 네이바는 절벽에서 뛰어내려 타바투이호에 물을 공급하는 조용한 강으로 변했다'

In [252]:
wiki_qa_df['question'][2053]

'작가 강신향은 어느 회사에서 일했어?'

In [148]:
wiki_qa_df['answer'][2053]

'김소진 (소설가)'

In [250]:
wiki_qa_df['context'][2053]

'1964년 강원도 철원군 김화읍에서 태어나 1967년에 서울로 이주하여 성장했다. 미아초등학교, 서울보성중학교, 서라벌고등학교를 거쳐 서울대학교 영문과를 졸업했으며, 1990년 한겨레에 입사하여 신문기자로 일했다.\n\n1991년 경향신문 신춘문예에 〈쥐잡기〉가 당선되어 등단하였다. 1993년 《열린 사회와 그 적들》(솔출판사, 89-85062-32-8), 1995년 《장석조네 사람들》, 1995년 《고아떤 뺑덕어멈》(솔출판사, 89-85062-97-2) 등 단편 소설집과 장편 소설을 연달아 간행, 같은 세대 작가들 사이에서 일약 주목받는 위치에 올라섰다.\n\n1995년 한겨레를 퇴사하고 전업 작가를 선언했으며, 1996년에 《자전거 도둑》, 《양파》(세계사, 89-338-0082-4)를 출간한 이후에도 〈신풍근 배커리 약사(略史)〉, 〈눈 속에 묻힌 검은 항아리〉 등의 단편을 꾸준히 발표하며 필력을 과시했다. 1996년 문화의 날에 제4회 젊은예술가상을 수상하였다.\n\n1997년 2월에는 계간 한국문학의 편집위원으로 참여했으나, 얼마 지나지 않아 동년 3월에 췌장암 판정을 받아 신촌 세브란스병원에 입원했고 그 다음 달인 4월 22일 동서한방병원에서 별세했다.\n\n유족으로는 1993년에 결혼한 소설가 함정임과 아들 김태형이 있으며, 대표적인 작품으로는 단편소설집 《열린 사회와 그 적들》, 장편소설 《장석조네 사람들》(1995), 단편소설집 《자전거 도둑》(1996), 창작 동화 《열한 살의 푸른 바다》(1996)가 있는데 이들 중 《자전거 도둑》이 1997년 SBS 단막극 《70분 드라마》에서 드라마화됐는데   비교적 서술구조에 충실한 각색을 했는데도 연출의 전반적인 표현방식이 서툴러 원작이 갖는 무게를 효과적으로 전달하는 데  실패했다.'

In [251]:
wiki_qa_df['answer'][2053] in wiki_qa_df['context'][2053]

False

In [183]:
wiki_qa_df['context'][50] + wiki_qa_df['context'][51]

'아놀드 알로이스 슈워제네거(Arnold Alois Schwarzenegger,  /ˈʃwɔrtsənɛɡər/ ,  aɐnɔlt aloʏs ʃvaɐtsənɛɡɐ , 아놀드 시바제네거, 1947년 7월 30일 ~ )는 오스트리아 태생의 미국의 배우, 보디빌더, 정치인이다.\n\n배우가 되기 전 1960년대부터 70년대에 보디빌더로 활동했으며,  38대 캘리포니아 주지사이기도하다. 케네디 대통령의 조카인 마리아 슈라이버와 결혼하여, 4명의 아이를 두고 있다, 보디빌더로서는 전설적인 존재이며 그의 이름 자체가 근육질의 대명사로 쓰이기도 한다.\n\n오스트리아 태생의 헐리웃 근육질 배우. 20살에 미국으로 건너와서 보디빌딩대회 미스터 올림피아를 7번이나 우승했고, 전성기는 <코난> 으로 80년대 초반부터 시작되어 지금까지도 속편이 계속 만들어지고 있는 <터미네이터>, 샤론스톤과 공연한 <토탈리콜>, <터미네이터2> 와 <트루 라이즈>에서 인기의 정점을 찍었었다. 스탤론과 비슷한 나이, 이미지로 항상 비교되던 액션배우. 47년생 188센티.슈워제네거는 오스트리아의 작은 마을 탈에서, 오스트리아 헌병 부사관 출신으로 지역 경찰서장이었던 구스타프 슈바르체네거의 2남 중 막내로 태어났다. 15세 때인 1961년 보디빌더가 됐으며 1970년까지 미스터 유니버스 1위 5회, 1970년부터 1980년까지 미스터 올림피아 1위 7회 등 역대 최다 우승 타이틀을 얻었. 병역은 무거운 물건을 들어나를 기회가 많다는 이유로 전차병으로 이행했으며 실제로도 전차 조종 자격증을 보유하고 있다. 1968년 미국 이민 후 1983년 미국 시민이 되었다 . 오스트리아는 본래 이중국적을 허락하지 않아  다른 국가로 귀화한 오스트리아 시민권자들은 국적을 포기해야 하지만, 슈와제너거는 미국 시민권 취득 전 오스트리아 정부로부터 특별 허가를 받아 오스트리아 국적을 유지할 수 있었다. 당시 세계 최고의 보디빌더로 불리던 슈워제네거는 1973년 기네스북에 \'지구상에서 상체근육이 가장 잘 발달된 사람\'에 

In [271]:
qg(wiki_qa_df['answer'][50], wiki_qa_df['context'][50]+wiki_qa_df['context'][51], beam = 10, top_p=0.9, len_penalty=0.01)

'터미네이터는 어떤 기록을 세웠어?'

In [154]:
wiki_qa_df['context'][2053]

'1964년 강원도 철원군 김화읍에서 태어나 1967년에 서울로 이주하여 성장했다. 미아초등학교, 서울보성중학교, 서라벌고등학교를 거쳐 서울대학교 영문과를 졸업했으며, 1990년 한겨레에 입사하여 신문기자로 일했다.\n\n1991년 경향신문 신춘문예에 〈쥐잡기〉가 당선되어 등단하였다. 1993년 《열린 사회와 그 적들》(솔출판사, 89-85062-32-8), 1995년 《장석조네 사람들》, 1995년 《고아떤 뺑덕어멈》(솔출판사, 89-85062-97-2) 등 단편 소설집과 장편 소설을 연달아 간행, 같은 세대 작가들 사이에서 일약 주목받는 위치에 올라섰다.\n\n1995년 한겨레를 퇴사하고 전업 작가를 선언했으며, 1996년에 《자전거 도둑》, 《양파》(세계사, 89-338-0082-4)를 출간한 이후에도 〈신풍근 배커리 약사(略史)〉, 〈눈 속에 묻힌 검은 항아리〉 등의 단편을 꾸준히 발표하며 필력을 과시했다. 1996년 문화의 날에 제4회 젊은예술가상을 수상하였다.\n\n1997년 2월에는 계간 한국문학의 편집위원으로 참여했으나, 얼마 지나지 않아 동년 3월에 췌장암 판정을 받아 신촌 세브란스병원에 입원했고 그 다음 달인 4월 22일 동서한방병원에서 별세했다.\n\n유족으로는 1993년에 결혼한 소설가 함정임과 아들 김태형이 있으며, 대표적인 작품으로는 단편소설집 《열린 사회와 그 적들》, 장편소설 《장석조네 사람들》(1995), 단편소설집 《자전거 도둑》(1996), 창작 동화 《열한 살의 푸른 바다》(1996)가 있는데 이들 중 《자전거 도둑》이 1997년 SBS 단막극 《70분 드라마》에서 드라마화됐는데   비교적 서술구조에 충실한 각색을 했는데도 연출의 전반적인 표현방식이 서툴러 원작이 갖는 무게를 효과적으로 전달하는 데  실패했다.'

In [273]:
qg(wiki_qa_df['answer'][134], wiki_qa_df['context'][134]+wiki_qa_df['context'][135]+wiki_qa_df['context'][136]+wiki_qa_df['context'][137], beam = 25, top_p=0.8, len_penalty=0.01)

'리누스 토르발스가 발표한 GNU가 뭐야?'

In [269]:
wiki_qa_df['context'][137]

'1985년, 스톨먼은 GNU 선언문을 발표했다. 이는 유닉스에 대항하여 자유로운 대안을 만들기 위한 그의 의지와 동기를 역설한 것이었다. 그리고 얼마 안 있어 그는 비영리 기관인 자유 소프트웨어 재단을 설립했다.\n\n그리고 그는 1989년 일반 공중 사용 허가서(GPL) 내에 카피레프트의 개념을 적용하였다. 허드(Hurd) 커널을 제외하고, 대부분의 GNU 시스템이 거의 동시에 완성되었다.\n\n1991년, 리누스 토르발스는 GPL로 리눅스 커널을 발표했다. 이를 통해 완벽하게 기능하는 GNU 시스템인 GNU/리눅스 운영 체제가 탄생하게 되었다.'

In [117]:
wiki_qg_df['context'][695]

'1921년 평안남도 중화군 중화면 청학리(현 황해북도 중화군 중화읍)에서 태어나 1934년 1월 25일에 중화천주교회에서 메리놀선교회 소속 요셉 클먼 신부로 부터 영세를 받고, 소신학교(신학 고등학교)인 서울 동성고등학교를 다녔다. 1948년 3월 함경남도 덕원에 있는 덕원신학교를 다니다가, 1950년 1월 17일 남북 분단 이후 공산 정권하의 북한에서 박해를 피해 친구인 윤공회 와 함께 월남하여 서울 성신대학(지금의 가톨릭대학교 신학대학)에 편입했다. 남한으로 탈출하던 중 체포되어 황해도 해주 감옥에 수감되었으며, 가족들은 그대로 북한에 남아 있었다.'

In [235]:
wiki_qg_df['answer'][1]

'백남준'

In [237]:
qg(wiki_qg_df['answer'][1], wiki_qg_df['context'][1])

'1966년 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양의 건축, 음악사, 철학 등을 공부한 사람은?'

In [264]:
qg(wiki_qg_df['answer'][1], wiki_qg_df['context'][1], beam = 10, top_p=0.9, len_penalty=0.1)

"'아시아에서 온 문화테러리스트'라는 칭호를 얻은 예술가는?"

In [241]:
wiki_qg_df['context'][1]

'현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친일파인 아버지 백낙승과 어머니 조종희 사이의 3남 2녀 중 막내로 태어났다. 그후 종로구 창신동 197번지 소위 "큰대문집"에서 18세까지 살았다. 수송국민학교와 경기제1고등보통학교를 다니면서 피아니스트 신재덕에게 피아노 연주를, 작이건우]]에게 작곡을 각각 배웠다. 이때 한국이 낳은 작곡가 김순남을 사사했다. 1949년 그는 홍콩 로이덴 스쿨로 전학했으며, 한국 전쟁이 발발하기 이전 가족이 일본으로 이주했다. 그 후 일본으로 건너가 1952년 도쿄 대학교 문과부에 입학했다. 2년 후 미술사학 및 미학으로 전공을 정했지만, 실제로는 일본 당대의 작곡가 모로이 사부로, 미학자 노무라 요시오 등에게서 작곡과, 음악사학을 공부했다. 졸업 논문은 ‘아르놀트 쇤베르크 연구’이다.\n\n1956년 백남준은 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양의 건축, 음악사, 철학 등을 공부하였다. 뮌헨 대학교 입학 1년 후에는 프라이부르크 국립 음악 대학교로 옮겨 볼프강 포르트너 교수에게 배우지만, 곧 쇤베르크 이후 현대음악의 실험이 활발히 진행되던 다름슈타트 하기 강좌에 참여했다. 1958년 그 곳에서 현대음악가 존 케이지를 만나 그의 음악에 대한 파괴적 접근과 자유정신으로부터 깊은 영감을 얻었다. 이 영감은 "세계의 역사는 우리에게 알려준다. 주어진 게임에서 이길 수 없다면 규칙을 바꿔라" 라는 것으로 규정된다. 이후 1950년대부터 활발해지기 시작한 독일 라인 지역의 액션뮤직의 현장에서 백남준은 ‘아시아에서 온 문화테러리스트’(앨런 카프로)라고 불릴 정도의 탁월한 퍼포먼스 아티스트로 활약했다. 1959년 ‘존 케이지에게 보내는 경의’에서 음악적 콜라주와 함께 피아노를 부수는 퍼포먼스를 선보이는 것을 시작으로, 바이올린을 단숨에 파괴하거나(바이올린 솔로) 존 케이지가 착용한 넥타이를 잘라버리는 퍼포먼스(피아노 포르테를 위한 연습곡)가 특히 유명하다. 이 초기 퍼포

In [163]:
qg(wiki_qg_df['answer'][10], wiki_qg_df['context'][10], beam = 10, top_p=0.9)

'프린키피아의 저자는?'

In [155]:
wiki_qg_df['context'][10]

"《자연철학의 수학적 원리》(自然哲學- 數學的原理, Philosophiae Naturalis Principia Mathematica|필로소피아이 나투랄리스 프린키피아 마테마티카)는 서양의 과학 혁명을 집대성한 책의 하나이다. 줄여서 '프린키피아'(Principia)라고 불리기도 한다. 1687년에 나온 아이작 뉴턴의 세 권짜리 저작으로, 라틴어로 썼다.\n\n이 책에서 뉴턴은 고전 역학의 바탕을 이루는 뉴턴의 운동 법칙과 만유인력의 법칙을 기술하고 있다. 당시 요하네스 케플러가 천체의 운동에 대한 자료를 바탕으로 알아낸 케플러의 행성운동법칙을 뉴턴은 자신의 위 두 법칙들로써 증명해 낸다. 그는 이러한 일련의 작업을 통해서 코페르니쿠스에서 시작되어 케플러, 갈릴레오를 거치면서 이루어져 온 천문학의 혁명을 완성하는 한편, 갈릴레오 이후 데카르트, 하위헌스 등을 통해서 이루어져 온 근대 역학의 성공을 눈부시게 보여주고 있다.\n\n에드먼드 핼리도 이 책을 바탕으로 1530년, 1607년, 1682년에 나타났던 혜성들의 궤도를 계산해, 이 혜성 모두가 동일한 하나의 천체일 가능성이 높다는 사실을 발견했고 일정한 주기에 따라 1750년대 말에 다시 나타나리라고 예견했다. 뉴턴도 핼리도 죽은 뒤인 1758년에 수수께끼 같은 천체가 발견되었는데 그것이 다름 아닌 핼리 혜성이다. \n\n제1편은 운동에 관한 일반적 명제를 논술하였고, 제2편은 매질 속에서의 물체의 운동을 다루고, 마지막 제3편은 코페르니쿠스의 지동설, 케플러의 행성의 타원궤도 등의 행성의 운동을 증명하였다.\n\n뉴턴은 그의 이론을 기술하기 위해 미적분학을 역학에 적용하였지만, 이 책에서는 주로 기하학적인 증명 방법을 사용하고 미적분을 거의 사용하지 않고 있는데, 이는 당시의 사람들의 이해를 고려해서라고 한다.\n\n1687년에 초판, 1712년 증보 개정판, 그리고 1726년 제3판이 출간되었다."

In [74]:
aaa = []
ccc = []
qqq = []
iii = []

for i in range(len(wiki_qa_df)):
    a = wiki_qa_df['answer'][i]
    if a not in aaa:
        ccc.append(wiki_qa_df['context'][i])
        aaa.append(a)
        qqq.append(wiki_qa_df['question'][i])
        iii.append(wiki_qa_df['doc_id'][i])
wiki_qg_df = pd.DataFrame(data={'answer':aaa,'question':qqq,'context':ccc, 'doc_id':iii})

In [75]:
len(wiki_qg_df)

31738

In [76]:
wiki_qg_df.head()

,answer,question,context,doc_id
0,나라 목록,전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하는 문서의 ...,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",0
1,백남준,1966년 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양...,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,2
2,아오조라 문고,일본어판 구텐베르크 프로젝트로 불리는 일본의 인터넷 전자도서관 이름은?,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",3
3,데니스 리치,애플 컴퓨터는 누구의 업적을 기반으로 하고 있어?,켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발...,8
4,주기율표,원소를 구분하기 쉽게 성질에 따라 배열한 표가 뭐야?,"주기율표(週期律表, 주기률표, periodic table) 또는 주기표(週期表)는 ...",9


In [79]:
wiki_qg_df.to_csv('wiki_qg_df.csv', index=False)

In [78]:
wiki_qg_df['context'][3]

'켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발했고, 1971년 최초의 〈Unix Programmer\'s Manual〉을 썼다. 또한 C 언어를 개발한 후 브라이언 커니핸과 함께 〈C 프로그래밍 언어〉(The C Programming Language)를 기술했다. 커니핸과 〈C 프로그래밍 언어〉책을 썼기에 커니핸이 C 언어 개발에 참여한 것으로 종종 오해받으나 커니핸의 말에 따르면 자신은 C언어 개발에 참여하지 않았다고 한다.\n\nALTRAN, B언어, BCPL, Multics 등의 개발에도 영향을 끼친 것으로도 알려져 있다.\n\n1983년에 켄 톰프슨과 "범용 운영체제 이론개발, 특히 유닉스 운영체제의 구현에 대한 공로"로 튜링상을 수상했다.\n\n미국의 경제 전문지 \'비즈니스 인사이더\'에서는 \'현재의 애플 컴퓨터는 거의 모두 데니스 리치의 업적에 기반하고 있다\'이라며 그의 업적을 평가했다.  현재 애플 매킨토시의 OS X와 아이폰의 iOS는 모두 유닉스 운영체제를 기반으로 만들어져 있다.'

In [234]:
len(wiki_qg_df)

31738

In [1]:
import pandas as pd
wiki = pd.read_csv('/opt/ml/wiki_arg_post_processing.csv')

In [3]:
wiki['context'][0].index(wiki['answer'][0])

6

In [2]:
wiki

,answer,question,context,doc_id
0,나라 목록,세계 각 지역의 현황과 주권 승인 정보를 기재하는 문서의 이름은?,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",0
1,백남준,"""큰대문집""에서 18세까지 살고, 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 ...",현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,2
2,아오조라 문고,일본어판 구텐베르크 프로젝트로 불리는 일본의 인터넷 전자도서관 이름은?,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",3
3,데니스 리치,애플 컴퓨터는 누구의 업적을 기반으로 하고 있어?,켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발...,8
4,주기율표,원소를 구분하기 쉽게 성질에 따라 배열한 표가 뭐야?,"주기율표(週期律表, 주기률표, periodic table) 또는 주기표(週期表)는 ...",9
...,...,...,...,...
16525,타바투이호,네이바가 어느 호수에서 태어났어?,타바투이호의 형성과 관련하여 전해지는 전설이 있다.\n\n예전에 이 지역에는 아름답...,60594
16526,2022년 FIFA 월드컵 유럽 지역 예선 플레이오프,결승전에서 승리한 3개 팀은 언제 본선 진출이 가능한가?,2022년 FIFA 월드컵 유럽 지역 예선 플레이오프 진행 방식은 2019년 12월...,60597
16527,자주국방론,박정희 정부가 분쟁의 책임을 당사국에 국한시킨 주장이 뭐야?,"박정희 정부가 자주국방론을 내놓은 배경은 ""아시아의 공산주의 국가와 자유진영 국가의...",60598
16528,오키나와 현립 박물관·미술관,오키나와에 있는 최초의 복합 문화시설은?,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",60608


In [19]:
a = [1]*len(wiki)
b = ['b']*len(wiki)

In [25]:
answers = []
context = []
question = []

for i in range(len(wiki)):
    context.append(wiki['context'][i])
    answers.append({'answer_start':[wiki['context'][i].index(wiki['answer'][i])], 'text':[wiki['answer'][i]]})
    question.append(wiki['question'][i])
wiki_qg_df = pd.DataFrame(data={'__index_level_0__':a, 'answers':answers,'context':context,'document_id':a, 'id':b, 'question':question,'title':b})

In [29]:
wiki_qg_df.to_json('qg_train.json', orient='columns')

In [24]:
wiki_qg_df

,__index_level_0__,answer,context,document_id,id,question,title
0,1,"{'answer_start': [6], 'text': ['나라 목록']}","이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",1,b,세계 각 지역의 현황과 주권 승인 정보를 기재하는 문서의 이름은?,b
1,1,"{'answer_start': [433], 'text': ['백남준']}",현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,1,b,"""큰대문집""에서 18세까지 살고, 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 ...",b
2,1,"{'answer_start': [0], 'text': ['아오조라 문고']}","아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",1,b,일본어판 구텐베르크 프로젝트로 불리는 일본의 인터넷 전자도서관 이름은?,b
3,1,"{'answer_start': [429], 'text': ['데니스 리치']}",켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발...,1,b,애플 컴퓨터는 누구의 업적을 기반으로 하고 있어?,b
4,1,"{'answer_start': [0], 'text': ['주기율표']}","주기율표(週期律表, 주기률표, periodic table) 또는 주기표(週期表)는 ...",1,b,원소를 구분하기 쉽게 성질에 따라 배열한 표가 뭐야?,b
...,...,...,...,...,...,...,...
16525,1,"{'answer_start': [0], 'text': ['타바투이호']}",타바투이호의 형성과 관련하여 전해지는 전설이 있다.\n\n예전에 이 지역에는 아름답...,1,b,네이바가 어느 호수에서 태어났어?,b
16526,1,"{'answer_start': [0], 'text': ['2022년 FIFA 월드컵...",2022년 FIFA 월드컵 유럽 지역 예선 플레이오프 진행 방식은 2019년 12월...,1,b,결승전에서 승리한 3개 팀은 언제 본선 진출이 가능한가?,b
16527,1,"{'answer_start': [8], 'text': ['자주국방론']}","박정희 정부가 자주국방론을 내놓은 배경은 ""아시아의 공산주의 국가와 자유진영 국가의...",1,b,박정희 정부가 분쟁의 책임을 당사국에 국한시킨 주장이 뭐야?,b
16528,1,"{'answer_start': [0], 'text': ['오키나와 현립 박물관·미술...","오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",1,b,오키나와에 있는 최초의 복합 문화시설은?,b


In [ ]:
csv_dataset = load_dataset('dataframe', data_files='/opt/ml/qg_train.csv')